The first part is to convert dat file from Lidar prototype to csv file (This can be done by running the first cell)

In [2]:
import re
import csv
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from pathlib import Path

# -----------------------------
# CONFIG THE DAY (ALL you need to do in this block!!!)
# -----------------------------
DAYS = [
    "05-01-2026",
    "06-01-2026",
    "07-01-2026",
]

BASE_DIR = Path(".")          # root directory
OUT_BASE = Path("csvfiles")   # base output directory
OUT_BASE.mkdir(exist_ok=True)

def make_time_tags():
    return [f"{hh:02d}.{mm:02d}" for hh in range(24) for mm in (5, 35)]
float_re = re.compile(
    r"""^[\s+-]?(?:\d+\.?\d*|\.\d+)(?:[eE][+-]?\d+)?$"""
)

def is_number(tok: str) -> bool:
    return bool(float_re.match(tok))

def convert_one_time_tag(day: str, time_tag: str):
    DAT_DIR = BASE_DIR / f"{day}-DATfile"
    OUT_DIR = OUT_BASE / day
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    infile = DAT_DIR / f"{time_tag}.dat"
    outfile = OUT_DIR / f"{time_tag}.csv"

    if not infile.exists():
        print(f"[SKIP] {day} {time_tag}")
        return

    rows = []
    with infile.open("r", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split()

            # Expect exactly 5 numeric columns
            if len(parts) == 5 and all(is_number(p) for p in parts):
                rows.append([float(p) for p in parts])

    if not rows:
        print(f"[WARN] Empty {day} {time_tag}")
        return

    with outfile.open("w", newline="") as f:
        w = csv.writer(f)
        w.writerow([
            "analog",
            "analog_sterr",
            "photon_counting",
            "pc_sterr",
            "overflow_info",
        ])
        w.writerows(rows)

    print(f"[OK] {day} {time_tag}: {len(rows)} rows → {outfile}")

for day in DAYS:
    print(f"\n=== Processing {day} ===")
    for TIME_TAG in make_time_tags():
        convert_one_time_tag(day, TIME_TAG)



=== Processing 05-01-2026 ===
[OK] 05-01-2026 00.05: 2000 rows → csvfiles/05-01-2026/00.05.csv
[OK] 05-01-2026 00.35: 2000 rows → csvfiles/05-01-2026/00.35.csv
[OK] 05-01-2026 01.05: 2000 rows → csvfiles/05-01-2026/01.05.csv
[OK] 05-01-2026 01.35: 2000 rows → csvfiles/05-01-2026/01.35.csv
[OK] 05-01-2026 02.05: 2000 rows → csvfiles/05-01-2026/02.05.csv
[OK] 05-01-2026 02.35: 2000 rows → csvfiles/05-01-2026/02.35.csv
[OK] 05-01-2026 03.05: 2000 rows → csvfiles/05-01-2026/03.05.csv
[OK] 05-01-2026 03.35: 2000 rows → csvfiles/05-01-2026/03.35.csv
[OK] 05-01-2026 04.05: 2000 rows → csvfiles/05-01-2026/04.05.csv
[OK] 05-01-2026 04.35: 2000 rows → csvfiles/05-01-2026/04.35.csv
[OK] 05-01-2026 05.05: 2000 rows → csvfiles/05-01-2026/05.05.csv
[OK] 05-01-2026 05.35: 2000 rows → csvfiles/05-01-2026/05.35.csv
[OK] 05-01-2026 06.05: 2000 rows → csvfiles/05-01-2026/06.05.csv
[OK] 05-01-2026 06.35: 2000 rows → csvfiles/05-01-2026/06.35.csv
[OK] 05-01-2026 07.05: 2000 rows → csvfiles/05-01-2026/07.0

In [76]:
AfterPulse = pd.read_csv("AfterPulse.csv")

As you can see, all of the files in selected "DAYS" folder will be change to csv file automatically 
But you need to have raw (dat file) first!!!

After that, you need to open csv file in order to automatically calculate Normalized Range Square (What we want)!

In [6]:
CSV_BASE = Path("csvfiles")

day_dfs = {}

for day in DAYS:
    day_dir = CSV_BASE / day
    if not day_dir.exists():
        print(f"[SKIP] {day} not found")
        continue

    dfs = []
    for csv_path in sorted(day_dir.glob("*.csv")):
        df = pd.read_csv(csv_path)
        df["day"] = day
        df["time_tag"] = csv_path.stem
        dfs.append(df)

    if dfs:
        day_dfs[day] = pd.concat(dfs, ignore_index=True)
        print(f"[OK] {day}: {len(day_dfs[day])} rows")


[OK] 05-01-2026: 92000 rows


In [8]:
#This is what we have got (You can open other day in the day_dfs["dd-mm-yyyy"] to see what's inside!) Cool!
day_dfs["05-01-2026"].head()

,analog,analog_sterr,photon_counting,pc_sterr,overflow_info,day,time_tag
0,28.9665,0.166826,176.1600,0.899132,0.0,05-01-2026,00.05
1,67.2555,0.400356,195.0520,0.864446,0.0,05-01-2026,00.05
2,274.8830,2.351700,114.0360,1.378620,0.0,05-01-2026,00.05
3,473.2520,0.070407,1.2828,0.143848,0.0,05-01-2026,00.05
4,473.4570,0.000000,0.0000,0.000000,0.0,05-01-2026,00.05


Next this is the parameters which you can change "overlap_r1_m" and "overlap_r2_m" (Changing others are unnecessary, but you can still try :))

In [12]:
#parameter
#note: blend region shold be wider than overlap about 50 unit to make a smooth line
config = {
    "bin_width_ns": 25,
    "bin_spacing_m": 3.75,
    "prf_hz": 20,
    "dead_time_ns": 3.06,
    "bg_start_m": 3000,
    "bg_end_m": 5621.25,
    "overlap_r1_m": 200,
    "overlap_r2_m": 300,
    "shift_search_bins": 20,
    "afterpulse_provided": True,
    "k_scale": 0.064021849,
    "b_offset": 0,
}
bin_spacing_m = 3.75
range_m = 2000


In [39]:
def add_bin_and_snr_day(df, *, bin_spacing_m, config=None):
    df = df.copy()

    # per-timestamp bin index
    df["bin_index"] = df.groupby(["day", "time_tag"]).cumcount()
    df["range_m"] = df["bin_index"] * bin_spacing_m

    # SNR
    df["SNR_analog"] = df["analog"] / df["analog_sterr"]
    df["SNR_Photon"] = df["photon_counting"] / df["pc_sterr"]

    # photon per bin
    if config is not None:
        df["photon_per_bin"] = (
            df["photon_counting"] * config["bin_width_ns"] * 1e-3
        )

    # ---- reorder columns ----
    main_cols = [
        "day",
        "time_tag",
        "bin_index",
        "range_m",
        "analog",
        "analog_sterr",
        "photon_counting",
        "pc_sterr",
        "overflow_info",
        "SNR_analog",
        "SNR_Photon",
        "photon_per_bin",
    ]

    # keep metadata columns (day, time_tag, timestamp, etc.)
    meta_cols = [c for c in df.columns if c not in main_cols]

    df = df[main_cols + meta_cols]

    return df

#call function
for day in list(day_dfs.keys()):
    day_dfs[day] = add_bin_and_snr_day(
        day_dfs[day],
        bin_spacing_m=bin_spacing_m,
        config=config
    )

In [84]:
# FULL PIPELINE (per-day, per-timestamp)
# 1) Load CSVs into day_dfs
# 2) Add bin_index, range_m, SNR, photon_per_bin
# 3) Add background correction + afterpulse + deadtime correction
# 4) (Optional) Reorder columns

import numpy as np
import pandas as pd
import re
from pathlib import Path

# -----------------------------
# CONFIG YOU MUST HAVE
# -----------------------------
# DAYS = ["05-01-2026", "06-01-2026", "07-01-2026"]
# config = {
#   "bin_width_ns": ...,
#   "dead_time_ns": ...,
#   "bg_start_m": ...,
#   "bg_end_m": ...,
# }
# bin_spacing_m = 3.75
#
# AfterPulse (optional):
# - DataFrame with column "afterpulse_raw"
#   AfterPulse = pd.read_csv("afterpulse.csv") or similar

CSV_BASE = Path("csvfiles")


# -----------------------------
# HELPERS
# -----------------------------
def add_bin_and_snr_day(df, *, bin_spacing_m, config=None):
    df = df.copy()
    df.columns = df.columns.str.strip()

    # bin_index resets per timestamp
    df["bin_index"] = df.groupby(["day", "time_tag"]).cumcount()
    df["range_m"] = df["bin_index"] * bin_spacing_m

    df["SNR_analog"] = df["analog"] / df["analog_sterr"]
    df["SNR_Photon"] = df["photon_counting"] / df["pc_sterr"]

    if config is not None:
        df["photon_per_bin"] = df["photon_counting"] * config["bin_width_ns"] * 1e-3

    # (optional) reorder core columns to the front
    main_cols = [
        "day",
        "time_tag",
        "bin_index",
        "range_m",
        "analog",
        "analog_sterr",
        "photon_counting",
        "pc_sterr",
        "overflow_info",
        "SNR_analog",
        "SNR_Photon",
        "photon_per_bin",
    ]
    
    keep_cols = [c for c in df.columns if c not in main_cols]
    df = df[[c for c in main_cols if c in df.columns] + keep_cols]

    return df


def add_bg_afterpulse_deadtime_day(df, *, config, AfterPulse=None):
    df = df.copy()
    df.columns = df.columns.str.strip()

    required = ["range_m", "analog", "photon_per_bin", "day", "time_tag"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"add_bg_afterpulse_deadtime_day: missing columns {missing}")

    bin_width_s = config["bin_width_ns"] * 1e-9
    dead_time_s = config["dead_time_ns"] * 1e-9

    # create columns (safe defaults)
    for col in [
        "analog_bg_corr",
        "Photon_per_bin_bg_corr",
        "afterpulse_raw",
        "afterpulse_counts_per_bin",
        "photon_APcorr_counts",
        "photon_deadtime_counts",
        "photon_deadtime_corr",
    ]:
        if col not in df.columns:
            df[col] = np.nan

    # Preload afterpulse raw array if provided
    ap_raw = None
    if AfterPulse is not None:
        if isinstance(AfterPulse, pd.DataFrame):
            if "afterpulse_raw" not in AfterPulse.columns:
                raise KeyError("AfterPulse DataFrame must have column 'afterpulse_raw'")
            ap_raw = AfterPulse["afterpulse_raw"].to_numpy()
        else:
            # allow passing a 1D array-like
            ap_raw = np.asarray(AfterPulse)

    # process per timestamp group
    for (day, time_tag), g_idx in df.groupby(["day", "time_tag"]).groups.items():
        g = df.loc[g_idx]

        bg_mask = (g["range_m"] >= config["bg_start_m"]) & (g["range_m"] <= config["bg_end_m"])
        bg_row = g.loc[bg_mask]
        if bg_row.empty:
            # no bg region for this timestamp
            continue

        analog_bg_mean = bg_row["analog"].mean()
        photon_bg_mean = bg_row["photon_per_bin"].mean()

        df.loc[g_idx, "analog_bg_corr"] = g["analog"] - analog_bg_mean
        df.loc[g_idx, "Photon_per_bin_bg_corr"] = g["photon_per_bin"] - photon_bg_mean

        # Afterpulse correction
        if ap_raw is not None:
            if "bin_index" not in g.columns:
                raise KeyError("Need 'bin_index' to align AfterPulse by bin. Run add_bin_and_snr_day first.")

            bi = g["bin_index"].to_numpy().astype(int)
            valid = (bi >= 0) & (bi < len(ap_raw))

            ap_vals = np.full(len(g), np.nan)
            ap_vals[valid] = ap_raw[bi[valid]]

            df.loc[g_idx, "afterpulse_raw"] = ap_vals
            df.loc[g_idx, "afterpulse_counts_per_bin"] = ap_vals * config["bin_width_ns"] * 1e-3

            df.loc[g_idx, "photon_APcorr_counts"] = (
                df.loc[g_idx, "Photon_per_bin_bg_corr"] - df.loc[g_idx, "afterpulse_counts_per_bin"]
            )
        else:
            # no afterpulse profile => AP-corrected = bg-corrected
            df.loc[g_idx, "photon_APcorr_counts"] = df.loc[g_idx, "Photon_per_bin_bg_corr"]

        # Deadtime correction
        x = df.loc[g_idx, "photon_APcorr_counts"].to_numpy()

        # stage 1
        photon_deadtime_counts = x / (1 - x * dead_time_s / bin_width_s)

        # validity
        ratio = photon_deadtime_counts * dead_time_s / bin_width_s

        photon_deadtime_corr = np.where(
            ratio < 1,
            photon_deadtime_counts / (1 - ratio),
            np.nan
        )

        df.loc[g_idx, "photon_deadtime_counts"] = photon_deadtime_counts
        df.loc[g_idx, "photon_deadtime_corr"] = photon_deadtime_corr

    return df


# -----------------------------
# 1) LOAD CSVs INTO day_dfs
# -----------------------------
day_dfs = {}

for day in DAYS:
    day_dir = CSV_BASE / day
    if not day_dir.exists():
        print(f"[SKIP] {day} not found")
        continue

    dfs = []
    for csv_path in sorted(day_dir.glob("*.csv")):
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        df["day"] = day
        df["time_tag"] = csv_path.stem
        dfs.append(df)

    if dfs:
        day_dfs[day] = pd.concat(dfs, ignore_index=True)
        print(f"[OK] {day}: {len(day_dfs[day])} rows")


# -----------------------------
# 2) ADD BIN/RANGE/SNR/PHOTON_PER_BIN
# 3) ADD BG + AFTERPULSE + DEADTIME CORRECTION
# -----------------------------
for day in list(day_dfs.keys()):
    day_dfs[day] = add_bin_and_snr_day(
        day_dfs[day],
        bin_spacing_m=bin_spacing_m,
        config=config
    )

    day_dfs[day] = add_bg_afterpulse_deadtime_day(
        day_dfs[day],
        config=config,
        AfterPulse=AfterPulse  # set to None if you don't have it
    )

print("\nDone. Example columns:")
print(list(day_dfs.keys())[0], day_dfs[list(day_dfs.keys())[0]].columns.tolist())


[OK] 05-01-2026: 92000 rows

Done. Example columns:
05-01-2026 ['day', 'time_tag', 'bin_index', 'range_m', 'analog', 'analog_sterr', 'photon_counting', 'pc_sterr', 'overflow_info', 'SNR_analog', 'SNR_Photon', 'photon_per_bin', 'analog_bg_corr', 'Photon_per_bin_bg_corr', 'afterpulse_raw', 'afterpulse_counts_per_bin', 'photon_APcorr_counts', 'photon_deadtime_counts', 'photon_deadtime_corr']


Check if this already added "bin_index", "range_m", "SNR_analog", "SNR_Photon", "photon_per_bin"

In [88]:
# day_dfs["05-01-2026"].head()

,day,time_tag,bin_index,range_m,analog,analog_sterr,photon_counting,pc_sterr,overflow_info,SNR_analog,SNR_Photon,photon_per_bin,analog_bg_corr,Photon_per_bin_bg_corr,afterpulse_raw,afterpulse_counts_per_bin,photon_APcorr_counts,photon_deadtime_counts,photon_deadtime_corr
0,05-01-2026,00.05,0,0.00,28.9665,0.166826,176.1600,0.899132,0.0,173.633007,195.922289,4.40400,25.248878,4.397529,0.0,0.0,4.397529,9.523770,NaN
1,05-01-2026,00.05,1,3.75,67.2555,0.400356,195.0520,0.864446,0.0,167.989240,225.638154,4.87630,63.537878,4.869829,0.0,0.0,4.869829,12.056033,NaN
2,05-01-2026,00.05,2,7.50,274.8830,2.351700,114.0360,1.378620,0.0,116.886933,82.717500,2.85090,271.165378,2.844429,0.0,0.0,2.844429,4.363679,9.366416
3,05-01-2026,00.05,3,11.25,473.2520,0.070407,1.2828,0.143848,0.0,6721.670745,8.917747,0.03207,469.534378,0.025599,0.0,0.0,0.025599,0.025679,0.025760
4,05-01-2026,00.05,4,15.00,473.4570,0.000000,0.0000,0.000000,0.0,inf,NaN,0.00000,469.739378,-0.006471,0.0,0.0,-0.006471,-0.006466,-0.006461


In [70]:
# def plot_one_timestamp_save(
#     df_t,
#     *,
#     day: str,
#     time_tag: str,
#     xcol="range_m",
#     ycol="photon_counting",
#     out_base="Pict",
#     dpi=300,
#     figsize=(7, 5),
#     xscale="linear",
#     yscale="linear",
# ):
#     tttt = str(time_tag).replace(".", "").replace(":", "")  # "00.05" -> "0005"
#     out_dir = Path(out_base) / f"{day}-{tttt}"
#     out_dir.mkdir(parents=True, exist_ok=True)

#     out_path = out_dir / f"{ycol}_vs_{xcol}.png"

#     plt.figure(figsize=figsize)
#     plt.plot(df_t[xcol], df_t[ycol])

#     plt.xlabel(xcol)
#     plt.ylabel(ycol)
#     plt.title(f"{day} {time_tag}: {ycol} vs {xcol}")
#     plt.xscale(xscale)
#     plt.yscale(yscale)
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     plt.savefig(out_path, dpi=dpi)
#     plt.close()

#     return out_path


# all_saved = []

# #loop through each day
# for day in DAYS:
#     if day not in day_dfs:
#         print(f"[SKIP] day_dfs has no day={day}")
#         continue

#     # ensure range_m etc exist
#     day_dfs[day] = add_bin_and_snr_day(
#         day_dfs[day],
#         bin_spacing_m=bin_spacing_m,
#         config=config
#     )

#     df_day = day_dfs[day]
#     time_tags = sorted(df_day["time_tag"].unique())

#     print(f"\n=== {day}: {len(time_tags)} timestamps ===")

#     for time_tag in time_tags:
#         df_t = df_day[df_day["time_tag"] == time_tag].copy()

#         if df_t.empty:
#             print(f"[SKIP] {day} {time_tag} empty")
#             continue

#         # optional: sort for nicer line plot
#         if "range_m" in df_t.columns:
#             df_t = df_t.sort_values("range_m")

#         out_path1 = plot_one_timestamp_save(
#             df_t,
#             day=day,
#             time_tag=time_tag,
#             xcol="range_m",
#             ycol="photon_counting",
#             out_base="Pict",
#         )
#         out_path2 = plot_one_timestamp_save(
#             df_t,
#             day=day,
#             time_tag=time_tag,
#             xcol="range_m",
#             ycol="analog",
#             out_base="Pict",
#         )
        
#         all_saved.extend([out_path1, out_path2])


# print(f"\nSaved total {len(all_saved)} figures across {len(DAYS)} days.")



=== 05-01-2026: 46 timestamps ===
[SKIP] day_dfs has no day=06-01-2026
[SKIP] day_dfs has no day=07-01-2026

Saved total 92 figures across 3 days.


In [74]:
# def plot_one_timestamp_twin_save(
#     df_t,
#     *,
#     day: str,
#     time_tag: str,
#     xcol="range_m",
#     y1col="photon_counting",
#     y2col="analog",
#     out_base="Pict",
#     dpi=300,
#     figsize=(5, 5),
#     xlim=None,
#     y1lim=None,
#     y2lim=None,
# ):
#     """
#     Plot twin-y figure:
#       Left axis  -> y1col (photon)
#       Right axis -> y2col (analog)

#     Saved to: Pict/dd-mm-yyyy-tttt/
#     """

#     # folder name
#     tttt = str(time_tag).replace(".", "").replace(":", "")
#     out_dir = Path(out_base) / f"{day}-{tttt}"
#     out_dir.mkdir(parents=True, exist_ok=True)

#     out_path = out_dir / f"{y1col}_and_{y2col}_vs_{xcol}.png"

#     fig, ax1 = plt.subplots(figsize=figsize)

#     # ---- Left axis (Photon) ----
#     ax1.plot(df_t[xcol], df_t[y1col], color="tab:blue", label=y1col)
#     ax1.set_xlabel("Range (m)", fontsize=14)
#     ax1.set_ylabel("Photon rate", color="tab:blue", fontsize=14)
#     ax1.tick_params(axis="y", labelcolor="tab:blue")
#     ax1.grid(True, alpha=0.3)

#     if xlim is not None:
#         ax1.set_xlim(*xlim)
#     if y1lim is not None:
#         ax1.set_ylim(*y1lim)

#     # ---- Right axis (Analog) ----
#     ax2 = ax1.twinx()
#     ax2.plot(df_t[xcol], df_t[y2col], color="tab:red", label=y2col)
#     ax2.set_ylabel("Analog signal", color="tab:red", fontsize=14)
#     ax2.tick_params(axis="y", labelcolor="tab:red")

#     if y2lim is not None:
#         ax2.set_ylim(*y2lim)

#     # ---- Title & save ----
#     plt.title(f"{day} {time_tag}: Photon & Analog vs Range", fontsize=16)
#     plt.tight_layout()
#     plt.savefig(out_path, dpi=dpi)
#     plt.close()

#     return out_path
    
# all_saved = []

# for day in DAYS:
#     if day not in day_dfs:
#         print(f"[SKIP] day_dfs has no day={day}")
#         continue

#     # ensure range_m, bin_index, SNR exist
#     day_dfs[day] = add_bin_and_snr_day(
#         day_dfs[day],
#         bin_spacing_m=bin_spacing_m,
#         config=config
#     )

#     df_day = day_dfs[day]
#     time_tags = sorted(df_day["time_tag"].unique())

#     print(f"\n=== {day}: {len(time_tags)} timestamps ===")

#     for time_tag in time_tags:
#         df_t = df_day[df_day["time_tag"] == time_tag].copy()

#         if df_t.empty:
#             print(f"[SKIP] {day} {time_tag} empty")
#             continue

#         # sort by range for clean plot
#         df_t = df_t.sort_values("range_m")

#         # ---- twin-axis plot ----
#         out_path = plot_one_timestamp_twin_save(
#             df_t,
#             day=day,
#             time_tag=time_tag,
#             xcol="range_m",
#             y1col="photon_counting",
#             y2col="analog",
#             out_base="Pict",
#             # optional limits:
#             # xlim=(0, 15000),
#             # y1lim=(0, 5e6),
#             # y2lim=(0, 4096),
#         )

#         all_saved.append(out_path)

# print(f"\nSaved total {len(all_saved)} twin-axis figures across {len(DAYS)} days.")



=== 05-01-2026: 46 timestamps ===
[SKIP] day_dfs has no day=06-01-2026
[SKIP] day_dfs has no day=07-01-2026

Saved total 46 twin-axis figures across 3 days.


In [90]:
def add_glue_merge_day(df, *, config):
    """
    For each (day, time_tag):
      - compute k_scale & b_offset from overlap region regression
      - compute analog_scaled_for_glue
      - cosine blend weight_w
      - merged_counts_per_bin, range2_corrected_counts, range2_norm
      - overlap ratio mean/std (store as columns)
    """
    df = df.copy()
    df.columns = df.columns.str.strip()

    required = [
        "day","time_tag","range_m",
        "analog_bg_corr","Photon_per_bin_bg_corr",
        "photon_APcorr_counts",
    ]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"add_glue_merge_day: missing columns {missing}")

    # output columns
    for col in [
        "k_scale","b_offset","r2_overlap",
        "analog_scaled_for_glue",
        "weight_w",
        "merged_counts_per_bin",
        "range2_corrected_counts",
        "range2_norm",
        "ratio_mean_overlap",
        "ratio_std_overlap",
    ]:
        if col not in df.columns:
            df[col] = np.nan

    # blend region
    blend_r1_m = config["overlap_r1_m"] - 100
    blend_r2_m = config["overlap_r2_m"] + 100

    # per timestamp
    for (day, time_tag), g_idx in df.groupby(["day","time_tag"]).groups.items():
        g = df.loc[g_idx]

        # overlap mask for regression
        m_overlap = (
            (g["range_m"] >= config["overlap_r1_m"]) &
            (g["range_m"] <= config["overlap_r2_m"])
        )
        go = g.loc[m_overlap]

        if go.empty:
            continue

        # regression x=analog_bg_corr, y=Photon_per_bin_bg_corr
        x = go["analog_bg_corr"].to_numpy(dtype=float)
        y = go["Photon_per_bin_bg_corr"].to_numpy(dtype=float)

        # drop NaNs
        ok = np.isfinite(x) & np.isfinite(y)
        if ok.sum() < 2:
            continue

        slope, intercept = np.polyfit(x[ok], y[ok], 1)  # slope=k_scale, intercept=b_offset

        # R^2
        yhat = slope * x[ok] + intercept
        ss_res = np.sum((y[ok] - yhat) ** 2)
        ss_tot = np.sum((y[ok] - np.mean(y[ok])) ** 2)
        r2 = np.nan if ss_tot == 0 else 1 - ss_res / ss_tot

        df.loc[g_idx, "k_scale"] = slope
        df.loc[g_idx, "b_offset"] = intercept
        df.loc[g_idx, "r2_overlap"] = r2

        # scaled analog
        df.loc[g_idx, "analog_scaled_for_glue"] = slope * g["analog_bg_corr"] + intercept

        # cosine blend weights
        r = g["range_m"].to_numpy(dtype=float)
        w = np.zeros_like(r, dtype=float)
        w[r > blend_r2_m] = 1.0
        m = (r >= blend_r1_m) & (r <= blend_r2_m)
        if blend_r2_m > blend_r1_m:
            w[m] = 0.5 * (1.0 - np.cos(np.pi * (r[m] - blend_r1_m) / (blend_r2_m - blend_r1_m)))

        df.loc[g_idx, "weight_w"] = w

        # merged
        df.loc[g_idx, "merged_counts_per_bin"] = (
            (1.0 - w) * df.loc[g_idx, "analog_scaled_for_glue"].to_numpy(dtype=float)
            + w * g["Photon_per_bin_bg_corr"].to_numpy(dtype=float)
        )

        # range^2 corrected
        df.loc[g_idx, "range2_corrected_counts"] = df.loc[g_idx, "merged_counts_per_bin"] * (r ** 2)

        # normalized per timestamp
        mx = np.nanmax(df.loc[g_idx, "range2_corrected_counts"].to_numpy(dtype=float))
        if np.isfinite(mx) and mx != 0:
            df.loc[g_idx, "range2_norm"] = df.loc[g_idx, "range2_corrected_counts"] / mx

        # ratio stats in overlap region (use photon_APcorr_counts per your snippet)
        denom = go["photon_APcorr_counts"].to_numpy(dtype=float)
        numer = (slope * go["analog_bg_corr"].to_numpy(dtype=float) + intercept)

        ok2 = np.isfinite(numer) & np.isfinite(denom) & (denom != 0)
        if ok2.sum() > 0:
            ratio = numer[ok2] / denom[ok2]
            df.loc[g_idx, "ratio_mean_overlap"] = np.mean(ratio)
            df.loc[g_idx, "ratio_std_overlap"] = np.std(ratio, ddof=1) if ok2.sum() > 1 else 0.0

    return df

for day in list(day_dfs.keys()):
    day_dfs[day] = add_bin_and_snr_day(
        day_dfs[day],
        bin_spacing_m=bin_spacing_m,
        config=config
    )

    day_dfs[day] = add_bg_afterpulse_deadtime_day(
        day_dfs[day],
        config=config,
        AfterPulse=AfterPulse
    )

    day_dfs[day] = add_glue_merge_day(
        day_dfs[day],
        config=config
    )



In [98]:
day_dfs["05-01-2026"].head()

,day,time_tag,bin_index,range_m,analog,analog_sterr,photon_counting,pc_sterr,overflow_info,SNR_analog,...,k_scale,b_offset,r2_overlap,analog_scaled_for_glue,weight_w,merged_counts_per_bin,range2_corrected_counts,range2_norm,ratio_mean_overlap,ratio_std_overlap
0,05-01-2026,00.05,0,0.00,28.9665,0.166826,176.1600,0.899132,0.0,173.633007,...,0.024338,4.256087,0.535032,4.870583,0.0,4.870583,0.000000,0.000000,1.002239,0.047687
1,05-01-2026,00.05,1,3.75,67.2555,0.400356,195.0520,0.864446,0.0,167.989240,...,0.024338,4.256087,0.535032,5.802444,0.0,5.802444,81.596870,0.000031,1.002239,0.047687
2,05-01-2026,00.05,2,7.50,274.8830,2.351700,114.0360,1.378620,0.0,116.886933,...,0.024338,4.256087,0.535032,10.855591,0.0,10.855591,610.626997,0.000233,1.002239,0.047687
3,05-01-2026,00.05,3,11.25,473.2520,0.070407,1.2828,0.143848,0.0,6721.670745,...,0.024338,4.256087,0.535032,15.683409,0.0,15.683409,1984.931411,0.000757,1.002239,0.047687
4,05-01-2026,00.05,4,15.00,473.4570,0.000000,0.0000,0.000000,0.0,inf,...,0.024338,4.256087,0.535032,15.688398,0.0,15.688398,3529.889523,0.001346,1.002239,0.047687


In [92]:
day = "05-01-2026"

summary = (
    day_dfs[day]
    .groupby("time_tag")[["k_scale","b_offset","r2_overlap","ratio_mean_overlap","ratio_std_overlap"]]
    .first()
)

print(summary)


           k_scale  b_offset  r2_overlap  ratio_mean_overlap  \
time_tag                                                       
00.05     0.024338  4.256087    0.535032            1.002239   
00.35     0.007229  4.431531    0.524546            1.002310   
01.05     0.017310  4.342812    0.599734            1.001355   
01.35     0.067303  4.064380    0.606059            1.001371   
02.05     0.014557  4.394354    0.581818            1.001195   
02.35     0.039306  4.197524    0.575167            1.001554   
03.05     0.008818  4.428326    0.546697            1.001848   
03.35     0.021375  4.306415    0.570523            1.001525   
04.05     0.007528  4.445789    0.520387            1.002272   
04.35     0.026352  4.266012    0.536107            1.001870   
05.05     0.020225  4.308831    0.541961            1.002044   
05.35     0.016652  4.287495    0.553957            1.002318   
06.05     0.034013  4.235518    0.539793            1.001677   
06.35     0.024383  4.292989    0.531425

In [ ]:
from pathlib import Path

def save_timestamp_csv(
    df_t,
    *,
    day: str,
    time_tag: str,
    out_base="Pict",
    suffix="processed",
):
    """
    Save processed CSV for one timestamp to:
      Pict/dd-mm-yyyy-tttt/dd-mm-yyyy-tttt_processed.csv
    """
    tttt = time_tag.replace(".", "").replace(":", "")
    out_dir = Path(out_base) / f"{day}-{tttt}"
    out_dir.mkdir(parents=True, exist_ok=True)

    out_path = out_dir / f"{day}-{tttt}_{suffix}.csv"
    df_t.to_csv(out_path, index=False)

    return out_path
